# Strands AgentでModel Context Protocol (MCP)をツールとして利用する

## 概要
[Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction)は、アプリケーションが大規模言語モデル（LLM）にコンテキストを提供する方法を標準化するオープンプロトコルです。Strands AI SDKはMCPと連携し、エージェントの機能を外部ツールやサービスで拡張できます。

MCPは、エージェントと追加ツールを提供するMCPサーバー間の通信を可能にします。Strands Agent SDKには、MCPサーバーへの接続とツール利用のための組み込みサポートが含まれています。

この例では、Strands AgentでMCPツールを利用する方法を紹介します。利用するMCPサーバーは[AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)で、AWSドキュメントへのアクセス、検索、レコメンデーション機能を提供します。このMCPサーバーには主に以下の3つの機能があります：

- **ドキュメントの取得**: AWSドキュメントページを取得し、Markdown形式に変換
- **ドキュメント検索**: 公式検索APIを利用したAWSドキュメントの検索
- **レコメンデーション**: AWSドキュメントページの関連コンテンツを提案



## エージェント詳細
<div style="float: left; margin-right: 20px;">
    
|機能                |説明                                               |
|--------------------|---------------------------------------------------|
|利用機能            |MCPツール                                          |
|エージェント構成    |シングルエージェントアーキテクチャ                 |

</div>

## アーキテクチャ

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## 主な特徴
* **シングルエージェントアーキテクチャ**: この例では、1つのエージェントがMCPツールと連携します
* **MCPツール**: エージェントにMCPツールを統合

## セットアップと前提条件

### 前提条件
* Python 3.10以上
* AWSアカウント
* Amazon BedrockでAnthropic Claude 3.7が有効

それでは、Strands Agentのための必要なパッケージをインストールしましょう。

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt

### 依存パッケージのインポート

次に、依存パッケージをインポートします。

In [ ]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### stdioトランスポートでMCPサーバーに接続する

MCPの[トランスポート](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports)は、クライアントとサーバー間の通信の基盤を提供します。メッセージの送受信の仕組みを担います。現在、MCPには3つの標準トランスポート実装があります:

- **標準入出力（stdio）**: 標準入力・出力ストリームを通じて通信します。ローカル統合やコマンドラインツールに便利です。
- **Streamable HTTP**: 以前のHTTP+SSEトランスポートの代替です。Streamable HTTPトランスポートでは、サーバーは複数のクライアント接続を処理できる独立したプロセスとして動作します。HTTP POSTやGETリクエストを利用し、必要に応じてServer-Sent Events（SSE）で複数のサーバーメッセージをストリーミングできます。
- **SSE**: HTTPベースのMCPサーバーでServer-Sent Eventsトランスポートを使うレガシー方式です。  

コマンドラインツールやローカル統合、シェルスクリプトで使う場合はstdioを、柔軟かつ効率的な通信やリソース消費を抑えたい場合はStreamable HTTPを使うのが推奨されます。

独自の用途に合わせて**カスタムトランスポート**も実装可能です。


それでは、stdioトランスポートでMCPサーバーに接続してみましょう。まず、`MCPClient`クラスを使って[AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)に接続します。このサーバーはAWSドキュメントへのアクセス、検索、レコメンデーション機能を提供します。

In [ ]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### エージェント設定と実行

次に、先ほど作成した`stdio_mcp_client`オブジェクトから利用可能なツールを取得し、エージェントの設定を行います。ツールの一覧は`list_tools_sync`メソッドで取得できます。

その後、エージェントに質問を投げてみましょう。

In [ ]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

### Streamable HTTPでMCPサーバーに接続する

次は、Streamable HTTPトランスポートでMCPサーバーに接続してみましょう。まず、Streamable HTTPトランスポートを使ったシンプルなMCPサーバーを起動します。

この例では独自のMCPサーバーを作成します。アーキテクチャは以下のようになります。

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [ ]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

それでは、`streamable-http`サーバーをスレッドで起動しましょう。

In [ ]:
thread = threading.Thread(target=main)
thread.start()

#### Streamable HTTPクライアントをエージェントに統合

次に、`streamablehttp_client`を使ってこのサーバーとシンプルなエージェントを連携させます。

In [ ]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### エージェント設定と実行

次に、先ほど作成した`streamable_http_mcp_client`オブジェクトから利用可能なツールを取得し、エージェントの設定を行います。ツールの一覧は`list_tools_sync`メソッドで取得できます。

その後、エージェントに質問を投げてみましょう。

In [ ]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

### ツールの直接呼び出し

通常、ツールはユーザーリクエストに応じてエージェントが呼び出しますが、MCPツールを直接呼び出すことも可能です。これは複数ツールを組み合わせたワークフローなどで便利です。

In [ ]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

MCPサーバーツールを呼び出す際、長時間実行を避けるために`read_timeout_seconds`を指定することもできます。

In [ ]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

### 複数のMCPサーバーとの連携

Strands Agentsでは、1つのエージェントで複数のMCPサーバーと連携し、`max_parallel_tools`（同時利用可能なツール数の上限）などのツール設定も可能です。以下の例でこの設定を紹介します。

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

このエージェントでは、再びAWS Documentation MCPサーバーを利用し、さらに[AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/)（AWS Cloud Development KitのベストプラクティスやCDK Nagによるセキュリティチェックなどを支援）も利用します。

まず、2つのMCPサーバーにstdioトランスポートで接続します。

In [ ]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### MCPサーバーでエージェントを作成

次に、両方のMCPサーバーから取得したツールでエージェントを作成します。

In [ ]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

### おめでとうございます！

このノートブックでは、Strands AgentでMCPサーバーに接続し、2種類のMCPトランスポート（stdioとStreamable HTTP）を使う方法を学びました。また、1つのエージェントで複数のMCPサーバーを連携する方法も紹介しました。次は、エージェントで異なるモデルを使う方法を見ていきましょう。